---
title: Single-cell scatter plot
author: Zafer Kosar
format:
    html:
        code-fold: true
        code-summary: "Show code"
---

In [1]:
# Core scverse libraries
import anndata as ad

# data manipulation
import polars as pl
import scanpy as sc

# ggplot but interactive and python
from lets_plot import (
    LetsPlot,
    aes,
    element_blank,
    element_line,
    element_text,
    geom_blank,
    geom_jitter,
    geom_point,
    geom_violin,
    gggrid,
    ggplot,
    ggsize,
    ggtb,
    guide_colorbar,
    guides,
    labs,
    layer_tooltips,
    scale_color_continuous,
    scale_color_gradient,
    scale_color_hue,
    scale_color_viridis,
    theme,
    theme_classic,
    geom_segment,
    arrow,
    element_rect,
    guide_legend,
    scale_shape,
)

LetsPlot.setup_html()

from typing import TYPE_CHECKING, Literal

from lets_plot.plot.core import PlotSpec
from lets_plot import *


import inspect

In [2]:
# read the sampel data
adata = sc.read("data/pbmc3k_pped.h5ad")

In [3]:
def theme_scatter(func):
    def modifier(*args, **kwargs):
        plot = func(*args, **kwargs)
        plot += (
            theme_classic()
            + theme(
                axis_text_x=element_blank(),
                axis_text_y=element_blank(),
                axis_ticks_y=element_blank(),
                axis_ticks_x=element_blank(),
                text=element_text(color="#1f1f1f", family="Arial", size=12, face="bold"),
                title=element_text(color="#1f1f1f", family="Arial"),
                axis_title_x=element_text(color="#3f3f3f", family="Arial", size=18),
                axis_title_y=element_text(color="#3f3f3f", family="Arial", size=18),
                legend_text=element_text(color="#1f1f1f", size=10, face="plain"),
                axis_line=element_blank(),
            )
            + ggsize(800, 600)
            + scale_shape(guide=guide_legend(nrow=3))
        )

        if kwargs.get("arrow_axis", True) is True:
            arrow_size = kwargs.get("arrow_size", 0.25)
            plot += theme(
                axis_title_x=element_text(hjust=arrow_size / 2),
                axis_title_y=element_text(hjust=arrow_size / 2),
            )

        return plot

    return modifier

In [4]:
@theme_scatter
def scatter(
    data,
    key: Literal["leiden", "louvain"] | str = "leiden",
    *,
    dimensions: Literal["umap", "pca", "tsne"] = "umap",
    size: float = 0.8,
    interactive: bool = False,
    color_low: str = "#ffffff",
    color_high: str = "#377eb8",
    arrow_axis: bool = True,
    arrow_size: float = 0.25,
    arrow_color: str = "#3f3f3f",
) -> PlotSpec:
    # Handling Data tpyes
    if not isinstance(data, sc.AnnData):
        raise ValueError("data must be an AnnData object")

    key_col = key

    # get the coordinates of the cells in the dimension reduced space
    frame = pl.from_numpy(
        data.obsm[f"X_{dimensions}"], schema=[f"{dimensions}1", f"{dimensions}2"]
    ).with_columns(pl.Series("ID", data.obs_names))

    # -------------------------- IF IT IS A CLUSTER --------------------------
    if key in ["leiden", "louvain"]:  # if it is a clustering
        key_col = "Cluster"  # update the key column name if it is a cluster
        frame = frame.with_columns(
            pl.Series("ID", data.obs_names), pl.Series(key_col, data.obs[key])
        )
        # cluster scatter
        scttr = (
            ggplot(data=frame)
            + geom_point(
                aes(x=f"{dimensions}2", y=f"{dimensions}1", color=key_col),
                size=size,
                tooltips=layer_tooltips(["ID", key_col]),
            )
            + scale_color_hue()
            + labs(
                x=f"{dimensions}2".upper(), y=f"{dimensions}1".upper()
            )  # UMAP1 and UMAP2 rather than umap1 and umap2 etc.,
            + scale_shape(guide=guide_legend(nrow=3))
        )
    # -------------------------- IF IT IS A GENE --------------------------
    elif key in data.var_names:  # if it is a gene
        # adata.X is a matrix , axis0 is cells, axis1 is genes
        # find the index of the gene
        index = data.var_names.get_indexer(
            data.var_names[data.var_names.str.startswith(key)]
        )  # get the index of the gene
        frame = frame.with_columns(
            pl.Series("ID", data.obs_names),
            pl.Series(key, data.X[:, index].flatten().astype("float64")),
        )
        scttr = (
            ggplot(data=frame)
            + geom_point(
                aes(x=f"{dimensions}2", y=f"{dimensions}1", color=key),
                size=size,
                tooltips=layer_tooltips(["ID", key]),
            )
            + scale_color_continuous(low=color_low, high=color_high)
            + labs(
                x=f"{dimensions}2".upper(), y=f"{dimensions}1".upper()
            )  # UMAP1 and UMAP2 rather than umap1 and umap2 etc.,
        )
    # -------------------------- GEOM SEGMENT --------------------------
    if arrow_axis:
        x_max = frame.select(f"{dimensions}2").max().item()
        x_min = frame.select(f"{dimensions}2").min().item()
        y_max = frame.select(f"{dimensions}1").max().item()
        y_min = frame.select(f"{dimensions}1").min().item()

        # find total difference between the max and min for both axis
        x_diff = x_max - x_min
        y_diff = y_max - y_min

        # find the ends of the arrows
        xend = x_min + arrow_size * x_diff
        yend = y_min + arrow_size * y_diff

        # adjust bottom ends of arrows
        x_adjusted = x_min - x_diff * 0.005
        y_adjusted = y_min - y_diff * 0.005

        # X axis
        scttr += geom_segment(
            x=x_adjusted,
            y=y_min,
            xend=xend,
            yend=y_min,
            color=arrow_color,
            size=3,
            arrow=arrow(20),
        )
        # Y axis
        scttr += geom_segment(
            x=x_min,
            y=y_adjusted,
            xend=x_min,
            yend=yend,
            color=arrow_color,
            size=3,
            arrow=arrow(20),
        )

    # -------------------------- NOT A GENE OR CLUSTER --------------------------
    else:
        msg = f"'{key}' is not present in cluster names nor gene names"
        raise msg

    if interactive:
        return scttr + ggtb()
    else:
        return scttr

In [5]:
signature = inspect.signature(scatter)

In [6]:
def get_default_args(func):
    signature = inspect.signature(func)
    return {
        k: v.default
        for k, v in signature.parameters.items()
        if v.default is not inspect.Parameter.empty
    }


get_default_args(scatter)

{}

In [7]:
print(scatter.__kwdefaults__)

None


## Test with Cluster [leiden]

In [8]:
scatter(adata, interactive=True, size=1) + scale_color_brewer(palette="Set2")

In [9]:
(
    scatter(
        adata,
        interactive=True,
        size=0.8,
    )
    + ggsize(600, 400)
).to_png("scatter.png")

'c:\\Users\\zafi_\\packages\\cellestia\\scatter.png'

In [10]:
(
    scatter(
        adata,
        interactive=True,
        size=0.8,
    )
    + ggsize(600, 400)
    + theme(
        plot_background=element_rect(color="red", size=4),
        legend_background=element_rect(color="blue", size=4),
    )
).to_png("scatter_rect.png")

'c:\\Users\\zafi_\\packages\\cellestia\\scatter_rect.png'

## Testing with a gene

In [11]:
scatter(adata, "MT-ND2", size=1.5, interactive=True)

### Customize Colors

In [12]:
scatter(adata, "MT-ND2", size=1.5, interactive=True, color_high="red", color_low="white")

In [13]:
for i in adata.obsm:
    print(i)

X_pca
X_umap


In [14]:
adata

AnnData object with n_obs × n_vars = 16680 × 2000
    obs: 'sample', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'n_genes', 'leiden'
    var: 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'hvg', 'leiden', 'leiden_colors', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [15]:
adata.obsm["X_umap"].shape

(16680, 2)

In [16]:
adata.obsm["X_pca"][:, :2].shape

(16680, 2)

In [17]:
adata.varm["PCs"].shape

(2000, 50)

In [18]:
adata.uns["pca"]

{'params': {'mask_var': 'highly_variable',
  'use_highly_variable': np.True_,
  'zero_center': np.True_},
 'variance': array([38.15803  , 31.635334 , 25.298174 , 16.957249 ,  9.686673 ,
         8.865039 ,  7.769258 ,  5.6994305,  5.1978235,  4.3297534,
         3.8270063,  3.539171 ,  3.2852602,  2.9820416,  2.835364 ,
         2.6374862,  2.4960923,  2.4309697,  2.3721688,  2.168274 ,
         2.0504627,  2.027751 ,  1.9263518,  1.755892 ,  1.7485383,
         1.6847519,  1.5971414,  1.5560592,  1.5314938,  1.4927095,
         1.4409993,  1.431091 ,  1.3803834,  1.3654335,  1.3360707,
         1.3212786,  1.3089134,  1.2980413,  1.2951055,  1.2794255,
         1.2674907,  1.2585825,  1.2423668,  1.241424 ,  1.23905  ,
         1.2297179,  1.2234632,  1.220291 ,  1.2102401,  1.2073023],
       dtype=float32),
 'variance_ratio': array([0.04706348, 0.03901849, 0.03120235, 0.02091479, 0.01194738,
        0.01093399, 0.00958247, 0.00702958, 0.00641091, 0.00534025,
        0.00472017, 0.00